In [1]:
# Import the necessary libraries

import numpy as np
import matplotlib.pyplot as plt

# mne import
from mne import Epochs, find_events, read_events, pick_types
from mne.io import Raw, RawArray

# pyriemann import
from pyriemann.estimation import Covariances
from pyriemann.tangentspace import TangentSpace
from pyriemann.utils.viz import plot_confusion_matrix
# cross validation

from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold

# lib to save the trained model
import pickle

# Get the dataset with the data to reproduce the real time acquisition of brain data - Pass the data for a RAW variable
raw_fname = '../data/record-[2012.07.19-11.24.02]_raw.fif'
raw = Raw(raw_fname, preload=True, verbose=False)

# replace baselining with high-pass
raw.filter(2, None, method='iir') 

# find events and generate epochs

event_fname = '../data/record-[2014.03.10-19.17.37]-eve.fif'
events = read_events(event_fname)
event_id = {'13 Hz': 2, '17 Hz': 4, '21 Hz': 3, 'resting-state': 1}

## Modificação de parâmetros Epoch e adicionado "Picks"
# Define time range (1 to labeled)
# Only works up to 151s in record-[2012.07.19-11.24.02]
tmin, tmax = -0., 1

# epochs = Epochs(raw, events, event_id, tmin=0, tmax=360.9, baseline=None)
epochs = Epochs(raw, events, event_id, tmin, tmax,  proj=False, baseline=None, preload=True, verbose=False)

# Extract data from Epochs object in (Trial, Channel, Sample) format
epochs_data = epochs.get_data()

In [2]:
# Make a pipeline with riemannian geometry models

clf = make_pipeline(Covariances(), TangentSpace(metric='riemann'), LogisticRegression())

# Get labels

labels = epochs.events[:, -1]

# Call de FIT function to train the model

clf.fit(epochs_data, labels)

# Save the model with pickle

Trained_Model = pickle.dumps(clf)

# Get the next data in "real time" and predict the label ['resting-state': 'resting', '13 Hz': 'forward', '21 Hz': 'right', '17 Hz': 'left']

Model = pickle.loads(Trained_Model)
# Transpoes matriz 3d to put samples in first index
epochs_data_transposed = epochs_data.transpose(2,1,0)
# Predict all samples
prediction = Model.predict(epochs_data_transposed)
# Predict only labeled samples (use 1s of time range)
prediction_labeled = Model.predict(epochs_data)

In [3]:
from numpy import linalg as la

def nearestPD(A):
    """Find the nearest positive-definite matrix to input
    A Python/Numpy port of John D'Errico's `nearestSPD` MATLAB code [1], which
    credits [2].
    [1] https://www.mathworks.com/matlabcentral/fileexchange/42885-nearestspd
    [2] N.J. Higham, "Computing a nearest symmetric positive semidefinite
    matrix" (1988): https://doi.org/10.1016/0024-3795(88)90223-6
    """

    B = (A + A.T) / 2
    _, s, V = la.svd(B)

    H = np.dot(V.T, np.dot(np.diag(s), V))

    A2 = (B + H) / 2

    A3 = (A2 + A2.T) / 2

    if isPD(A3):
        return A3

    spacing = np.spacing(la.norm(A))
    # The above is different from [1]. It appears that MATLAB's `chol` Cholesky
    # decomposition will accept matrixes with exactly 0-eigenvalue, whereas
    # Numpy's will not. So where [1] uses `eps(mineig)` (where `eps` is Matlab
    # for `np.spacing`), we use the above definition. CAVEAT: our `spacing`
    # will be much larger than [1]'s `eps(mineig)`, since `mineig` is usually on
    # the order of 1e-16, and `eps(1e-16)` is on the order of 1e-34, whereas
    # `spacing` will, for Gaussian random matrixes of small dimension, be on
    # othe order of 1e-16. In practice, both ways converge, as the unit test
    # below suggests.
    I = np.eye(A.shape[0])
    k = 1
    while not isPD(A3):
        mineig = np.min(np.real(la.eigvals(A3)))
        A3 += I * (-mineig * k**2 + spacing)
        k += 1

    return A3

def isPD(B):
    """Returns true when input is positive-definite, via Cholesky"""
    try:
        _ = la.cholesky(B)
        return True
    except la.LinAlgError:
        return False

In [11]:
# Tentativa de transformar as Epochs em matriz de covariancia para fazer o predict com dados de mesmo escopo
cov = Covariances().transform(epochs_data)

# Force cov matrix to be PSD
covPSD = []
for matrix in cov:
    matrix = nearestPD(matrix)
    covPSD.append(matrix)
covPSD = np.array(covPSD)

# Verify if matrix is PSD
allPD = True
for matrix in cov:
    isMatrixPD = isPD(matrix)
    if (not isMatrixPD):
        allPD = False

# Verifiy if have NaN in matrix
#print("is nan?: "+ str(np.any(np.isnan(cov))))
# Verify if all values are finite in matrix
#print("is finite?: "+ str(np.all(np.isfinite(cov))))
#
fitsInFloat = True
for matrix in cov:
    MatrixfitsInFloat = np.all(i < np.nextafter(0, 1) for i in matrix)
    if (not MatrixfitsInFloat):
        fitsInFloat = False
print("fits in float?: " + str(fitsInFloat))
# Predict
prediction = Model.predict(cov)

SyntaxError: unmatched ')' (<ipython-input-11-5513a8aaa02a>, line 25)